# sqlalchemy ORM

## Connect to database

In [1]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# Declare a Mapping

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users' 
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)  
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

In [6]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('password', String(), table=<users>), schema=None)

## Create a Schema

In [7]:
Base.metadata.create_all(engine)

2017-04-15 13:46:52,853 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-04-15 13:46:52,855 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 13:46:52,858 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-04-15 13:46:52,859 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 13:46:52,862 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-04-15 13:46:52,864 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 13:46:52,867 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2017-04-15 13:46:52,868 INFO sqlalchemy.engine.base.Engine ()
2017-04-15 13:46:52,870 INFO sqlalchemy.engine.base.Engine COMMIT


## Create an Instance of the Mapped Class¶

In [8]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
ed_user

<User(name='ed', fullname='Ed Jones', password='edspassword')>

## Creating a Session

In [14]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

## Adding and Updating Objects¶

In [19]:
ed_user = User(name='ed', fullname='Ed Jones', password='edspassword')
session.add(ed_user)
our_user = session.query(User).filter_by(name='ed').first()
ed_user is our_user

2017-04-15 13:57:31,143 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 13:57:31,146 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edspassword')
2017-04-15 13:57:31,151 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2017-04-15 13:57:31,153 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


True

In [20]:
session.commit()

2017-04-15 13:57:33,171 INFO sqlalchemy.engine.base.Engine COMMIT


## Rolling Back

In [21]:
fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)
fake_user in session

True

In [22]:
session.rollback()
fake_user in session

False

In [23]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2017-04-15 13:57:38,761 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-15 13:57:38,763 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2017-04-15 13:57:38,765 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', password='edspassword')>]

## Query

In [24]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', password='foobar'),
    User(name='mary', fullname='Mary Contrary', password='xxg527'),
    User(name='fred', fullname='Fred Flinstone', password='blah')])
session.commit()

2017-04-15 13:59:19,924 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 13:59:19,926 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'foobar')
2017-04-15 13:59:19,928 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 13:59:19,929 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'xxg527')
2017-04-15 13:59:19,931 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2017-04-15 13:59:19,932 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flinstone', 'blah')
2017-04-15 13:59:19,933 INFO sqlalchemy.engine.base.Engine COMMIT


### Query a table

In [27]:
for i in session.query(User).order_by(User.id):
    print(i.name, i.fullname)

2017-04-15 14:00:15,594 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-04-15 14:00:15,596 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2017-04-15 14:00:15,597 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query by sepcific columns

In [33]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2017-04-15 14:02:42,986 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2017-04-15 14:02:42,988 INFO sqlalchemy.engine.base.Engine ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flinstone


### Query the all rows and a column 

In [34]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2017-04-15 14:04:30,036 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users
2017-04-15 14:04:30,038 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='edspassword')> ed
<User(name='wendy', fullname='Wendy Williams', password='foobar')> wendy
<User(name='mary', fullname='Mary Contrary', password='xxg527')> mary
<User(name='fred', fullname='Fred Flinstone', password='blah')> fred


### Change the label

In [35]:
for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2017-04-15 14:04:47,611 INFO sqlalchemy.engine.base.Engine SELECT users.name AS name_label 
FROM users
2017-04-15 14:04:47,613 INFO sqlalchemy.engine.base.Engine ()
ed
wendy
mary
fred


### Make aliases

In [36]:
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')
for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2017-04-15 14:05:40,380 INFO sqlalchemy.engine.base.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.password AS user_alias_password 
FROM users AS user_alias
2017-04-15 14:05:40,383 INFO sqlalchemy.engine.base.Engine ()
<User(name='ed', fullname='Ed Jones', password='edspassword')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


### Slicing of the query

In [37]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2017-04-15 14:10:50,046 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2017-04-15 14:10:50,048 INFO sqlalchemy.engine.base.Engine (2, 1)
<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>


### `filter_by()`

In [38]:
for name, in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2017-04-15 14:11:41,900 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-15 14:11:41,902 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
ed


### `filter()`

In [39]:
for name in session.query(User.name).filter(User.fullname=='Ed Jones'):
    print(name)

2017-04-15 14:13:36,591 INFO sqlalchemy.engine.base.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2017-04-15 14:13:36,593 INFO sqlalchemy.engine.base.Engine ('Ed Jones',)
('ed',)


### `AND` of filters

In [40]:
for user in session.query(User).filter(User.name=='ed').filter(User.fullname=='Ed Jones'):
    print(user)

2017-04-15 14:14:59,016 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2017-04-15 14:14:59,018 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones')
<User(name='ed', fullname='Ed Jones', password='edspassword')>
